In [12]:
import pandas as pd
from functools import reduce

In [2]:
fs = '../ppi_ml/data/meta/euk_codes_ordered.txt'
species_ordered = [line.strip() for line in open(fs, 'r')]

In [28]:
# initialize empty df list
count_dfs = []
# set path to og data
path = '../ppi_ml/data/og_proteomes/nog_mapping/'
suffix = '.euNOG.diamond.mapping.2759'
# aggregate prot counts for each orthogroup in each proteome
for species in species_ordered:
    f = path+species+suffix
    df = pd.read_csv(f, sep = '\t')
    df = df[['ID', 'ProteinID']].drop_duplicates()
    dfg = df.groupby(['ID']).count()
    dfg.rename(columns={'ProteinID':species}, inplace=True)
    count_dfs.append(dfg)

final_df = pd.concat(count_dfs, axis=1, ignore_index=False)
final_df.fillna(0, inplace=True)
final_df = final_df.astype(int)
final_df['mean_family_size'] = final_df.mean(axis=1)
final_df['median_family_size'] = final_df.median(axis=1)
final_df = final_df.round(decimals=2).sort_values(by=['median_family_size'], ascending=False)
final_df.head()

,brart,caeel,dicdi,drome,human,mouse,nemve,pig,strpu,xenla,...,chlre,cocnu,maize,orysj,selml,sollc,soybn,wheat,mean_family_size,median_family_size
ID,,,,,,,,,,,,,,,,,,,,,
KOG1075,170,19,211,0,1,2,150,502,579,126,...,24,36,201,170,22,139,314,254,184.87,129.0
KOG0619,47,29,23,65,120,117,47,116,189,194,...,28,72,89,158,97,145,333,585,108.81,93.0
KOG1721,49,95,5,226,709,619,127,536,269,528,...,7,71,122,80,33,64,162,243,141.65,70.0
KOG0017,69,10,41,0,13,6,60,24,542,75,...,27,83,91,103,65,1482,394,328,293.58,67.0
KOG0504,70,18,14,25,97,88,50,78,231,126,...,24,61,67,155,26,58,158,490,82.42,64.0


In [29]:
len(final_df)

48414

In [30]:
outfile = '../ppi_ml/data/meta/avg_famsize_per_og.csv'
final_df.reset_index(inplace=True)
final_df.to_csv(outfile, index=False)